Algoritmos recursivos de grafos
===============================

**Autor:** Daniel R. Cassar



## Uma breve pausa



O que acontece quando o **valor padrão de um argumento** de uma função é um objeto mutável? Veja o código abaixo e reflita sobre isso.



In [ ]:
def minha_funcao(lista=[]):
    lista.append(1)
    return lista

print(minha_funcao([1,2,3]))
print(minha_funcao())
print(minha_funcao())
print(minha_funcao())

**Sugestão**: evite utilizar valores padrão que são objetos mutáveis de Python. Listas e dicionários são objetos mutáveis, mas inteiros, floats e strings não são.

**Sugestão 2**: evite alterar argumentos mutáveis dentro das funções para evitar problemas. Tanto listas quanto dicionários têm o método `copy` que pode ser utilizado para copiar o argumento de entrada e evitar alterá-lo no corpo da função.



## Busca em profundidade



### Problema a ser resolvido



<hr>

**Problema**: encontrar todos os vértices de um grafo que podem ser visitados a partir de um vértice inicial.

**Entradas**: um grafo representado por um dicionário e um vértice inicial.

**Saída**: lista contendo todos os vértices que podem ser visitados a partir de um vértice inicial.

<hr>



### Sobre o algoritmo



Assim como o algoritmo de **busca em largura** que estudamos em uma aula anterior, o algoritmo de **busca em profundidade** [1] também é um algoritmo para navegar grafos.

Esse algoritmo inicia sua busca em um vértice inicial e se &ldquo;aprofunda&rdquo; em cada ramo o máximo possível antes de voltar atrás e explorar outro ramo. Ele é outro algoritmo de travessia de grafos, junto com a busca em largura.

Diferente da busca em largura, a busca em profundidade não é capaz de resolver o problema de encontrar a menor distância entre dois vértices de um grafo. No entanto, a busca em largura é capaz de resolver outros problemas, como a ordenação topológica que será vista na próxima seção.



### Implementação com recursão



O algoritmo abaixo recebe um grafo e um vértice inicial e retorna a ordem de visitação dos vértices a partir do vértice inicial seguindo a estratégia da busca em profundidade.

**Caso base**: ocorre quando o vértice já foi visitado, ou seja, a função não precisa continuar a busca a partir dele.

**Passo recursivo**: ocorre quando a função encontra um vizinho ainda não visitado e chama a si mesma para continuar a busca a partir desse vizinho.

**Nota**: o argumento `visitados` dessa função é utilizado apenas durante o *passo recursivo*, por isso ele tem um valor padrão de `None`.



In [ ]:
def busca_em_profundidade(grafo, vertice, visitados=None):
    """Realiza busca em profundidade em um grafo."""

    if visitados is None:
        visitados = []

    # Caso base: se o vértice já foi visitado, a função não entra no if
    if vertice not in visitados:
        visitados.append(vertice)
        lista_vizinhos = grafo.get(vertice, [])

        for vizinho in lista_vizinhos:
            # Passo recursivo: encontrou um vizinho ainda não visitado
            visitados = busca_em_profundidade(grafo, vizinho, visitados)

    return visitados

**Teste**:



In [ ]:
from networkx import DiGraph, draw

grafo = {
    1: [2, 3],
    2: [4],
    3: [4],
    5: [6],
}

g = DiGraph(grafo)
draw(g, with_labels=True)

In [ ]:
vertice_inicial = 1

busca_em_profundidade(grafo, vertice_inicial)

### Implementação sem recursão



A implementação sem recursão da busca em profundidade usa a **estrutura de pilha** (último que entra é o primeiro que sai).



In [ ]:
def busca_em_profundidade_sem_recursao(grafo, vertice):

    pilha = [vertice]
    vertices_visitados = []

    while len(pilha) > 0:
        onde_estou = pilha.pop()
        if onde_estou not in vertices_visitados:
            vertices_visitados.append(onde_estou)
            vizinhos = grafo.get(onde_estou, [])
            pilha.extend(vizinhos)

    return vertices_visitados

**Teste**:



In [ ]:
vertice_inicial = 1

busca_em_profundidade_sem_recursao(grafo, vertice_inicial)

## Grafos cíclicos e acíclicos



Um grafo é chamado de **cíclico** se contém pelo menos um ciclo, ou seja, um caminho fechado em que um vértice é visitado mais de uma vez sem repetir arestas.

Um grafo que não tem nenhum ciclo é chamado de **acíclico**.

O grafo abaixo é um exemplo de grafo **direcional cíclico**. Observe que é possível sair do vértice $A$ e retornar nesse mesmo vértice seguindo um caminho fechado que não repete nenhuma aresta.



In [ ]:
grafo = {
    "A": ["B"],
    "B": ["C"],
    "C": ["D"],
    "D": ["E", "G", "C"],
    "E": ["A"],
    "F": ["C"],
}

g = DiGraph(grafo)
draw(g, with_labels=True)

## Ordenador topológico



### Problema a ser resolvido



Podemos ordenar os vértices de um **grafo direcional acíclico**. Ordenar, neste caso, significa listar os vértices de forma que todas as setas tenham sentido da esquerda para a direita. Veja um exemplo na imagem abaixo.

![img](https://i.imgur.com/Q3MA6dZ.png)

É possível que exista **mais de uma solução** para ordenar os vértices de um certo grafo. Isso não é um problema. Em geral, neste caso precisamos apenas encontrar uma solução qualquer, usualmente não precisamos encontrar *todas* as soluções.



### Sobre o algoritmo



O algoritmo que usaremos é um [*ordenador recursivo de grafos*](https://media2.giphy.com/media/v1.Y2lkPTc5MGI3NjExanJyYXFkZmNwam43NXV5MmJqZzlzdTg5dDBvb2czNHZ0NWdpbTY3YiZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/lXu72d4iKwqek/giphy.gif)!

Este algoritmo é baseado na busca em profundidade, mas em vez de começar de um vértice fornecido pelo usuário, ele garante que todos os vértices do grafo sejam explorados, mesmo que estejam em componentes desconexas.

**Passo recursivo**: ocorre quando a função encontra um vizinho ainda não visitado e chama a si mesma para continuar a busca a partir desse vizinho.

**Critério de parada**: ocorre quando todos os vizinhos do vértice atual já foram visitados, ou seja, não há mais chamadas recursivas a fazer.



In [ ]:
def ordenador_topologico(grafo):
    """Realiza ordenação topológica de um grafo."""

    visitados = []
    resultado = []

    def busca_em_profundidade(vertice):
        if vertice not in visitados:
            visitados.append(vertice)
            lista_vizinhos = grafo.get(vertice, [])

            for vizinho in lista_vizinhos:
                busca_em_profundidade(vizinho)

            resultado.append(vertice)

    for vertice in grafo:
        busca_em_profundidade(vertice)

    return list(reversed(resultado))

**Teste**:



In [ ]:
grafo = {
    0: [1, 2],
    1: [5, 2],
    2: [3],
    5: [3, 4],
    6: [1, 5],
}

ordenador_topologico(grafo)

Observe que a solução do algoritmo não é igual à solução apresentada na imagem. Isso vem do fato que podem existir diversas soluções válidas para a ordenação topológica de grafos. Novamente, quando utilizamos esse algoritmo estamos interessados em encontrar uma solução qualquer, não uma solução específica.



## XKCD relevante



![img](https://imgs.xkcd.com/comics/dfs.png)

`Imagem: DFS (XKCD) disponível em https://xkcd.com/761`



## Referências



1.  Artigo da Wikipédia sobre busca em profundidade [https://en.wikipedia.org/wiki/Depth-first_search](https://en.wikipedia.org/wiki/Depth-first_search)

2.  Fonte da imagem da ordenação topológica [https://imgur.com/Q3MA6dZ](https://imgur.com/Q3MA6dZ)

